In [1]:
import tensorflow as tf
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
options = {
    'seq_len': 128,
    'pred_len': 7,
    'using_cols': ['open', 'high', 'low', 'close', 'vol'],
}

d_k = int(256*5/8)
d_v = int(256*5/8)
batch_size = 24
n_heads = 12
ff_dim = int(256*5/8)
seq_len = options['seq_len']
pred_len = options['pred_len']

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## Time Embedding

In [4]:
time_cols_num = 4
class Time2Vector(nn.Module):
    def __init__(self, seq_len, device):
        super(Time2Vector, self).__init__()
        self.seq_len = seq_len

        self.weights_linear = torch.rand(self.seq_len, requires_grad=True).to(device)
        self.biad_linear = torch.rand(self.seq_len, requires_grad=True).to(device)

        self.weights_periodic = torch.rand(self.seq_len, requires_grad=True).to(device)
        self.bias_periodic = torch.rand(self.seq_len, requires_grad=True).to(device)

    def forward(self, inputs):
        x = torch.mean(inputs[:,:,:time_cols_num], dim=-1)
        time_linear = self.weights_linear * x + self.biad_linear
        time_linear = torch.unsqueeze(time_linear, dim=-1)

        time_periodic = torch.sin(x * self.weights_periodic + self.bias_periodic)
        time_periodic = torch.unsqueeze(time_periodic, dim=-1)
        return torch.cat([time_linear, time_periodic], dim=-1)

## TransformerEncoder + Attention

In [5]:
class StockModel(nn.Module):
    def __init__(self, time_embedding, transformer_encoder):
        super(StockModel, self).__init__()
        self.time_embedding = time_embedding
        self.transformer_encoder = transformer_encoder
        self.gap1d = nn.AvgPool1d(kernel_size=7*n_heads)
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.1)
        self.linear_mid = nn.Linear(seq_len, 64)
        self.linear_last = nn.Linear(64, pred_len)

    def forward(self, inputs):
        x = self.time_embedding(inputs)
        x = torch.cat([inputs, x], dim=-1)

        x = torch.cat([x for _ in range(n_heads)], dim=-1)
        x = self.transformer_encoder(x)

        x = self.gap1d(x)
        x = x.flatten(1)
        x = self.dropout1(x)
        x = F.relu(self.linear_mid(x))
        x = self.linear_last(x)

        return x

## Construct Model

In [16]:
time_embedding = Time2Vector(seq_len, device)
transformer_encoder = nn.TransformerEncoder(
    encoder_layer=nn.TransformerEncoderLayer(d_model=7*n_heads, nhead=n_heads,
                                             dim_feedforward=ff_dim,
                                             dropout=0.1,
                                             batch_first=True,
                                             device=device),
    num_layers=3,
)

stockmodel = StockModel(time_embedding, transformer_encoder).double().to(device)

## Training

In [22]:
def train(model, epoch, optimizer, criterion, train_dataloader, scheduler=None):
    print(f'[Train epoch: {epoch}]')
    if scheduler: scheduler.step()
    model.train()
    train_loss = 0
    train_mape = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_dataloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()

        optimizer.step()

        train_loss += loss.item()
        train_mape += torch.sum((targets - outputs).abs()/targets.abs())/targets.size(-1)
        total += targets.size(0)

    print(f'Train loss: {train_loss/total}, MAPE loss: {train_mape/total}')
    return train_loss/total, train_mape/total

def validate(model, epoch, criterion, valid_dataloader):
    print(f'Validation epoch: {epoch}')
    model.train()
    val_loss = 0
    val_mape = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(valid_dataloader):
        inputs, targets = inputs.to(device), targets.to(device)

        outputs = model(inputs)
        val_loss += criterion(outputs, targets).item()
        val_mape += torch.sum((targets - outputs).abs()/targets.abs())/targets.size(-1)
        total += targets.size(0)

    print(f'Validation loss: {val_loss/total}, MAPE loss: {val_mape/total}')
    return val_loss/total, val_mape/total

def test(model, epoch, criterion, test_dataloader):
    print(f'Test epoch: {epoch}')
    model.eval()
    test_loss = 0
    test_mape = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(test_dataloader):
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        test_loss += criterion(outputs, targets).item()
        test_mape += torch.sum((targets - outputs).abs()/targets.abs())/targets.size(-1)
        total += targets.size(0)

    print(f'Test loss: {test_loss/total}, MAPE loss: {test_mape/total}')
    return test_loss/total, test_mape/total

In [23]:
import time
import os
import torch.optim as op

criterion = nn.MSELoss()
optimizer = op.Adam(stockmodel.parameters(), lr=0.005, weight_decay=2e-5)

def fit(model, epoch, criterion, optimizer, train_dataloader, valid_dataloader, scheduler=None, file_name=None):
    train_result = []
    valid_result = []

    start_time = time.time()

    for i in range(epoch):
        train_loss, train_mape = train(model, i, optimizer, criterion, train_dataloader, scheduler)
        valid_loss, valid_mape = validate(model, i+1, criterion, valid_dataloader)
        train_result.append((train_loss, train_mape))
        valid_result.append((valid_loss, valid_mape))

        if file_name and not i%10:
            state = {'model': model.state_dict()}
            if not os.path.isdir(f'checkpoint_{i}'):
                os.mkdir(f'checkpoint_{i}')
            torch.save(state, f'./checkpoint_{i}/'+file_name)
            print(f'Model Saved! (time elapsed: {time.time() - start_time})')

    return train_result, valid_result

In [24]:
from bluesunb.preprocessing.stock_preprocessing import load_data_with_preprocessing
from torch.utils.data import DataLoader
file_path = os.path.abspath('../data/samsung-stock-data-2000-2021.csv')
(X_train, y_train), (X_valid, y_valid), (X_test, y_test), df = \
    load_data_with_preprocessing(file_path, options)


In [25]:
train_dataloader = DataLoader(list(zip(X_train, y_train)), batch_size=batch_size, num_workers=3)
valid_dataloader = DataLoader(list(zip(X_valid, y_valid)), batch_size=batch_size, num_workers=3)
test_dataloader = DataLoader(list(zip(X_test, y_test)), batch_size=batch_size, num_workers=3)

In [26]:
train_result, valid_result = fit(stockmodel, 40, criterion, optimizer, train_dataloader, valid_dataloader, file_name='tmp_model.pt')

[Train epoch: 0]


KeyboardInterrupt: 